<a href="https://colab.research.google.com/github/amalvarezme/AprendizajeMaquina/blob/main/7_TopicosAvanzados/Autoencoders/AutoencoderPCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, losses, Model
from tensorflow.keras.datasets import mnist
import numpy as np

# Load and prepare the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
scale = 0.4
x_train = x_train.astype('float32') / 255. + np.random.normal(scale=scale,size=x_train.shape)
x_test = x_test.astype('float32') / 255. + np.random.normal(scale=scale,size=x_test.shape)

# create training, validation, and testing sets
x_val = x_train[50000:]
y_val = y_train[50000:]
x_train = x_train[:50000]
y_train = y_train[:50000]
x_train = x_train[..., tf.newaxis]
x_val = x_val[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

print(x_train.shape,x_val.shape,x_test.shape,y_train.shape,y_val.shape,y_test.shape)

In [ ]:
y_train

In [ ]:
import matplotlib.pyplot as plt

#plot original images vs reconstructed images
def plot_mnist_autoencoder(x,xpred,cmap='gray',vmin=0,vmax=1):
  fig,ax = plt.subplots(2,x.shape[0],figsize=(8,1))
  for i,class_ in enumerate(range(x.shape[0])):
        ax[0,i].imshow(x[i],cmap=cmap,vmin=vmin,vmax=vmax)
        ax[0,i].set_xticks([])
        ax[0,i].set_yticks([])

        ax[1,i].imshow(xpred[i],cmap=cmap,vmin=vmin,vmax=vmax)
        ax[1,i].set_xticks([])
        ax[1,i].set_yticks([])
  plt.show()
  return

plot_mnist_autoencoder(x_train[:15],x_train[:15])

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

#plot images on latent space
def plot_mnist_2d(Z,y,images,img_w=28,img_h=28,zoom=0.5,cmap='jet'):
    fig, ax = plt.subplots(figsize=(5,5))
    plt.axis('off')
    for i in range(Z.shape[0]):
        #print('img',i+1,'/',Z.shape[0])
        image = images[i].reshape((img_w, img_h))
        im = OffsetImage(image, zoom=zoom,cmap=cmap)
        ab = AnnotationBbox(im, (Z[i,0], Z[i,1]), xycoords='data', frameon=False)
        ax.add_artist(ab)
        ax.update_datalim([(Z[i,0], Z[i,1])])
        ax.autoscale()
    plt.show()

In [ ]:
from sklearn.decomposition import PCA

#traditional PCA algorithm
red = PCA(n_components=2, random_state=123)
Z = red.fit_transform(x_train.reshape(x_train.shape[0],-1))
N = 500
plot_mnist_2d(Z[:N],y_train[:N],x_train[:N],img_w=28,img_h=28,zoom=0.3,cmap='gray')

In [ ]:
#custom autoencoder
class Autoencoder(Model):
    def __init__(self, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoding_dim = encoding_dim
        # Encoder layers
        self.encoder_input_layer = layers.Flatten()
        self.encoder_dense_layer = layers.Dense(encoding_dim, activation='relu')
        # Decoder layers will be initialized in build()
        self.decoder_dense_layer = None
        self.decoder_output_layer = None

    def build(self, input_shape):
        # Now that we have the input shape, initialize decoder layers
        self.decoder_dense_layer = layers.Dense(input_shape[1]*input_shape[2], activation='sigmoid')
        self.decoder_output_layer = layers.Reshape(input_shape[1:])
        super().build(input_shape)

    def call(self, inputs):
        x = self.encoder_input_layer(inputs)
        encoded = self.encoder_dense_layer(x)
        x = self.decoder_dense_layer(encoded)
        decoded = self.decoder_output_layer(x)
        return decoded


In [ ]:
# Instantiate the autoencoder
encoding_dim = 64
input_shape = (None, 28, 28, 1)
autoencoder = Autoencoder(encoding_dim)
autoencoder.build(input_shape)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
autoencoder.layers[1].get_weights()[0].shape

In [ ]:
# Define the loss object and the optimizer
tf.keras.backend.clear_session()

loss_object = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

# Define measures to track loss
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='val_loss')

@tf.function
def train_step(images):
    with tf.GradientTape() as tape:
        reconstructed = autoencoder(images, training=True)
        loss = loss_object(images, reconstructed)
    gradients = tape.gradient(loss, autoencoder.trainable_variables)
    optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables))
    train_loss(loss)

@tf.function
def test_step(images):
    reconstructed = autoencoder(images, training=False)
    t_loss = loss_object(images, reconstructed)
    test_loss(t_loss)

# Training loop
epochs = 20
batch_size = 64
# Prepare the dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(buffer_size=1024).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices(x_val).shuffle(buffer_size=128).batch(64)

for epoch in range(epochs):
    # Reset the metrics at the start of each epoch
    train_loss.reset_states()
    test_loss.reset_states()

    for images in train_dataset:
        train_step(images)

    for val_images in val_dataset:
        test_step(val_images)

    print(f'Epoch {epoch + 1}, '
          f'Loss: {train_loss.result()}, '
          f'Test Loss: {test_loss.result()}')
    if (epoch+1)%5 == 0:
      val_reconstructed = autoencoder(val_images, training=False)
      print(val_reconstructed.shape)
      plot_mnist_autoencoder(val_images,val_reconstructed)


In [ ]:
#latent space of trained autoencoder + PCA2D
from sklearn.manifold import TSNE
red = TSNE(n_components=2,perplexity=10,random_state=123,verbose=100)
N = 500
Z = red.fit_transform(autoencoder.layers[1](autoencoder.layers[0](x_val[:N])).numpy().reshape(N,-1))

In [ ]:
#plot 2D tsne from autoencoder latent space
plot_mnist_2d(Z[:N],y_val[:N],x_val[:N],img_w=28,img_h=28,zoom=0.3,cmap='gray')
plt.scatter(Z[:N,0],Z[:N,1],c=y_val[:N])
plt.colorbar()
plt.show()

In [ ]:
#compute inner product among basis
o_ = tf.linalg.matmul(autoencoder.layers[1].get_weights()[0],autoencoder.layers[1].get_weights()[0],transpose_a=True)
plt.pcolormesh(o_.numpy())
plt.colorbar()
plt.show()

In [ ]:
class DenseTransposeLayer(layers.Layer):
    def __init__(self, units, factor_o=0.1,activation=None, **kwargs):
        super(DenseTransposeLayer, self).__init__(**kwargs)
        self.units = units
        self.factor_o = factor_o
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape, self.units),
            initializer="random_normal",
            trainable=True,regularizer=tf.keras.regularizers.OrthogonalRegularizer(factor=self.factor_o),
            constraint=tf.keras.constraints.max_norm(1.)
        )
        #self.b1 = self.add_weight(
        #    shape=(self.units,), initializer="zeros", trainable=True)
        #self.b2 = self.add_weight(
        #    shape=(input_shape[-1],), initializer="zeros", trainable=True)

        super(DenseTransposeLayer, self).build(input_shape)

    def call(self, inputs):
        x = tf.linalg.matmul(inputs, self.w) #+ self.b1
        if self.activation is not None:
            x = self.activation(x)
        x = tf.linalg.matmul(x, tf.transpose(self.w)) #+ self.b2
        return x

In [ ]:
# Orthogonal Autoencoder with linear activation : PCA as keras-based NN

#custom autoencoder
class PCAutoencoder(Model):
    def __init__(self, encoding_dim,factor_o=0.1):
        super(PCAutoencoder, self).__init__()
        self.encoding_dim = encoding_dim
        self.factor_o=factor_o
        # Encoder layers
        self.encoder_input_layer = layers.Flatten()

        # Decoder layers will be initialized in build()
        self.encoder_decoder_transpose = DenseTransposeLayer(self.encoding_dim, factor_o=self.factor_o,activation='linear')
        self.decoder_output_layer = None

    def build(self, input_shape):
        # Now that we have the input shape, initialize decoder layers
        self.encoder_decoder_transpose.build(input_shape[1]*input_shape[2])
        self.decoder_output_layer = layers.Reshape(input_shape[1:])
        super().build(input_shape)


    def call(self, inputs):
        x = self.encoder_input_layer(inputs)
        x = self.encoder_decoder_transpose(x)
        decoded = self.decoder_output_layer(x)
        return decoded


In [ ]:
# Instantiate the autoencoder
encoding_dim = 64
input_shape = (None, 28, 28, 1)
factor_o = 0.1
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.1)
pcautoencoder = PCAutoencoder(encoding_dim,factor_o=factor_o)
pcautoencoder.build(input_shape)
pcautoencoder.compile(optimizer='adam', loss='mse')
pcautoencoder.summary()

In [ ]:
#pca Autoencoder Training
tf.keras.backend.clear_session()
@tf.function
def train_step(images):
    with tf.GradientTape() as tape:
        reconstructed = pcautoencoder(images, training=True)
        loss = loss_object(images, reconstructed)
    gradients = tape.gradient(loss, pcautoencoder.trainable_variables)
    optimizer.apply_gradients(zip(gradients, pcautoencoder.trainable_variables))
    train_loss(loss)

@tf.function
def test_step(images):
    reconstructed = pcautoencoder(images, training=False)
    t_loss = loss_object(images, reconstructed)
    test_loss(t_loss)

# Training loop
epochs = 20
batch_size = 64
# Prepare the dataset for training
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(buffer_size=1024).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices(x_val).shuffle(buffer_size=128).batch(64)

for epoch in range(epochs):
    # Reset the metrics at the start of each epoch
    train_loss.reset_states()
    test_loss.reset_states()

    for images in train_dataset:
        train_step(images)

    for val_images in val_dataset:
        test_step(val_images)

    print(f'Epoch {epoch + 1}, '
          f'Loss: {train_loss.result()}, '
          f'Test Loss: {test_loss.result()}')
    if (epoch+1)%5 == 0:
      val_reconstructed = pcautoencoder(val_images, training=False)
      print(val_reconstructed.shape)
      plot_mnist_autoencoder(val_images,val_reconstructed)


In [ ]:
#compute inner product among basis
o_ = tf.linalg.matmul(pcautoencoder.layers[1].get_weights()[0],pcautoencoder.layers[1].get_weights()[0],transpose_a=True)
plt.pcolormesh(o_.numpy())
plt.colorbar()
plt.show()

In [ ]:
plt.pcolormesh(autoencoder.layers[1].get_weights()[0])
plt.colorbar()
plt.show()